In [135]:
import torch
print(torch.__version__)
import time

1.8.1+cu101


In [ ]:
torch.cuda.is_available()


In [2]:
import louvain



/tmp/ipykernel_530/4063137415.py:1: DeprecationWarning: This package has been superseded by the `leidenalg` package and will no longer be maintained. Please upgrade to the `leidenalg` package.
  import louvain


In [ ]:
class copu:
    def __init__(self, cuda_device):
        self.cuda_device = cuda_device

    def copula_nonuniform_generator(n, mu, sigma):
        self.copula_nonuniform_generator = 
        
        tester = LDPTwoSampleTester(device)
generator_X = torch.distributions.multivariate_normal.MultivariateNormal(loc = copula_mean_1, covariance_matrix = sigma)
generator_Y = torch.distributions.multivariate_normal.MultivariateNormal(loc = copula_mean_2, covariance_matrix = sigma)
cdf_calculator = torch.distributions.normal.Normal(loc = 0.0, scale = 1.0)

test_results = torch.empty(nTests)
for rep in range(nTests):
    print(f"{rep+1}th run")
    
    data_x = cdf_calculator.cdf(generator_X.sample((n1,)))
    data_y = cdf_calculator.cdf(generator_Y.sample((n2,)))
  x <- rmvnorm(n, mean = mu, sigma = sigma)
  return(as.matrix(pnorm(x)))
}

In [136]:
class LDPTwoSampleTester:
    def __init__(self, cuda_device):
        self.cuda_device = cuda_device

    def run_test_conti_data(self, B, data_X, data_Y, kappa, alpha, gamma, discrete = False):
        '''
        '''  
        dataPrivatized = self.preprocess_conti_data(data_X, data_Y, kappa)
        n_1 = data_X.size(dim = 0)
        
 
        
        ustatOriginal = self.u_stat_twosample(dataPrivatized, n_1)
        print(f"original u-statistic:{ustatOriginal}")
        
        #permutation procedure
        permStats = torch.empty(B).to(self.cuda_device)
        
        for i in range(B):
            perm_stat_now = self.u_stat_twosample(
                dataPrivatized[torch.randperm(dataPrivatized.size(dim=0))],
                n_1).to(self.cuda_device)
            permStats[i] = perm_stat_now
            #print(perm_stat_now)
         
        
        p_value_proxy = (1 +
                         torch.sum(
                             torch.gt(input = permStats, other = ustatOriginal)
                         )
                        ) / (B + 1)
        
        
        print(f"p value proxy: {p_value_proxy}")
        return(p_value_proxy < gamma)#test result: TRUE = 1 = reject the null, FALSE = 0 = retain the null.
    
    def preprocess_conti_data(self, data_X, data_Y, kappa):
        data_X_binned = self.bin(data_X, kappa)
        data_Y_binned = self.bin(data_Y, kappa)
        
        dataCombined = torch.cat([data_X_binned, data_Y_binned], dim = 0)
        dataPrivatized = self.privatize_twosample(dataCombined, alpha)
        return(dataPrivatized)
        

    def bin(self, data, kappa): 
        ''' Only for continuous data'''
        
        # create designated number of intervals
        d = self.get_dimension(data)
     
        # 1. for each dimension, turn the continuous data into interval
        # each row now indicates a hypercube in [0,1]^d
        # the more the data is closer to 1, the larger the interval index.
        dataInterval = self.transform_bin_index(data = data, nIntervals = kappa)
        
        # 2. for each datapoint(row),
        #    turn the hypercube data into a multivariate data of (1, 2, ..., kappa^d)
        #    each row now becomes an integer.
        dataMultivariate = self.TransformMultivariate(
            dataInterval = dataInterval,
            nBin = kappa,
        )
        # 3. turn the indices into one-hot vectors
        dataOnehot = self.TransformOnehot(dataMultivariate, kappa**d)
        return(dataOnehot)
    
    def transform_bin_index(self, data, nIntervals):
        ''' Only for continuous data.
        for each dimension, transform the data in [0,1] into the interval index
        first interval = [0, x], the others = (y z]
        
        input arguments
            data: torch tensor object on GPU
            nIntervals: integer
        output
            dataIndices: torch tensor, dimension same as the input
        '''
        # create designated number of intervals
        d = self.get_dimension(data)
        breaks = torch.linspace(start = 0, end = 1, steps = nIntervals + 1).to(self.cuda_device) #floatTensor
        dataIndices = torch.bucketize(data, breaks, right = False) # ( ] form.
        dataIndices = dataIndices.add(
            dataIndices.eq(0)
        ) #move 0 values from the bin number 0 to the bin number 1        
        return(dataIndices)
    
    def TransformMultivariate(self, dataInterval, nBin):
        """Only for continuous and multivariate data ."""
        d = self.get_dimension(dataInterval)
        
        if d == 1:
            return(dataInterval.sub(1))
        else:
            exponent = torch.linspace(start = (d-1), end = 0, steps = d, dtype = torch.long)
            vector = torch.tensor(nBin).pow(exponent)
            return( torch.matmul( dataInterval.sub(1).to(torch.float), vector.to(torch.float).to(device) ).to(torch.long) )
    
    def TransformOnehot(self, dataMultivariate, newdim):
        return(
            torch.nn.functional.one_hot(
                dataMultivariate,
                num_classes = newdim)
        )
    
    def privatize_twosample(self, data, alpha = float("inf"), discrete_noise = False):
        ## assume the data is discrete by nature or has already been dicretized.
        n = data.size(dim = 0)
        dim = data.size(dim = 1) #kappa^d if conti data, d if discrete data
        print(f"noise dimension : {dim}")
        scale = torch.tensor(dim**(1/2))
        
        if alpha == float("inf"): #non-private case
            return( torch.mul(scale, data) )
        else: # private case
            if discrete_noise:
                noise = self.noise_discrete(n = n, dim = dim, alpha = alpha)
            else:
                noise = self.noise_conti(n = n, dim = dim, alpha = alpha)
        return(
            
            torch.add(
                input = noise.reshape(n, -1),
                alpha = scale,
                other = data
            )
        )
    
    def noise_conti(self, n, dim, alpha):
        #dim = kappa^d for conti data, d for discrete data
        unit_laplace_generator = torch.distributions.laplace.Laplace(
            torch.tensor(0.0).to(self.cuda_device),
            torch.tensor(2**(-1/2)).to(self.cuda_device)
        )
        laplace_samples = unit_laplace_generator.sample(sample_shape = torch.Size([n * dim]))
        scale = (8**(1/2) / alpha) * (dim**(1/2))
        laplace_samples = scale*laplace_samples
        print(f"noise variance: {torch.var(laplace_samples)}")
        return(laplace_samples)
    
  
        
    
    def noise_discrete(self, n, dim, alpha):
        #dim = kappa^d for conti data, d for discrete data
        param_geom = 1 - torch.exp(torch.tensor(-alpha / (2* (dim**(1/2)) )))
        n_noise =  n * dim
        geometric_generator = torch.distributions.geometric.Geometric(param_geom.to(self.cuda_device))
        noise = geometric_generator.sample((n_noise,)) - geometric_generator.sample((n_noise,))
        return(noise)
    
    def u_stat_twosample(self, data, n_1):
        n_2 = data.size(dim = 0) - n_1
        
        data_x = data[ :n_1, ]
        data_y = data[n_1: , ]
        
        # x only part
        u_x = torch.matmul(data_x, torch.transpose(data_x, 0, 1))
        u_x.fill_diagonal_(0)
        u_x = torch.sum(u_x) / (n_1 * (n_1 - 1))
        
        # y only part
        u_y = torch.matmul(data_y, torch.transpose(data_y, 0, 1))
        u_y.fill_diagonal_(0)
        u_y = torch.sum(u_y) / (n_2 * (n_2 - 1))

        # x, y part
        u_xy = torch.matmul(data_x, torch.transpose(data_y, 0, 1))
        u_xy = torch.sum(u_xy) * ( 2 / (n_1 * n_2) )
        return(u_x + u_y - u_xy)
    
    def get_dimension(self, data):
        if data.dim() == 1:
            return(1)
        elif data.dim() == 2:
            return( data.size(dim = 1) )
        else:
            return # we only use up to 2-dimensional tensor, i.e. matrix
        

# 2d coupula simulation

In [101]:
generator_X = torch.distributions.multivariate_normal.MultivariateNormal(loc = copula_mean_1, covariance_matrix = sigma)
generator_X.sample((10,))



tensor([[-0.2743, -0.3388],
        [-1.9330, -1.0486],
        [-0.1716, -1.4263],
        [-0.7701, -1.7462],
        [-1.3559, -0.3302],
        [ 0.1768, -1.0656],
        [ 0.0597,  0.8872],
        [-0.8105, -2.2033],
        [-1.0974, -0.9578],
        [-1.5348, -1.0953]], device='cuda:0')

In [129]:


####CHANGE HERE#####
n1 = 1000 
n2 = 1000
####################
kappa = 5 #number of bins
alpha = 0.3 #privacy level
gamma = 0.05 # significance level
nTests = 200 #number of tests for power estimation
B = 200 # number of permutations

start_time = time.time()

copula_mean_1 = torch.tensor([-1.0, -1.0]).to(device)
copula_mean_2 = torch.tensor([1.0, 1.0]).to(device)
sigma = torch.tensor([[1.0, 0.5], [0.5, 1.0]]).to(device)

tester = LDPTwoSampleTester(device)
generator_X = torch.distributions.multivariate_normal.MultivariateNormal(loc = copula_mean_1, covariance_matrix = sigma)
generator_Y = torch.distributions.multivariate_normal.MultivariateNormal(loc = copula_mean_2, covariance_matrix = sigma)
cdf_calculator = torch.distributions.normal.Normal(loc = 0.0, scale = 1.0)

test_results = torch.empty(nTests)
for rep in range(nTests):
    print(f"{rep+1}th run")
    
    data_x = cdf_calculator.cdf(generator_X.sample((n1,)))
    data_y = cdf_calculator.cdf(generator_Y.sample((n2,)))
    
    
    result_now = tester.run_test_conti_data(B, data_x, data_y,
                                            kappa, alpha, gamma, discrete = False
                                           )
    test_results[rep] = result_now
    print(f"result: {result_now}")
  
print( f"power estimate : { torch.sum(test_results)/nTests }" )
print( f"elapsed time: { time.time() - start_time }" )


1th run
noise dimension : 25
noise variance: 2227.487548828125
original u-statistic:61.441856384277344
p value proxy: 0.05970149114727974
result: False
2th run
noise dimension : 25
noise variance: 2204.18017578125
original u-statistic:-14.018186569213867
p value proxy: 0.711442768573761
result: False
3th run
noise dimension : 25
noise variance: 2225.160888671875
original u-statistic:-47.336326599121094
p value proxy: 0.9353233575820923
result: False
4th run
noise dimension : 25
noise variance: 2180.204833984375
original u-statistic:33.690189361572266
p value proxy: 0.16417910158634186
result: False
5th run
noise dimension : 25
noise variance: 2254.37255859375
original u-statistic:2.8982343673706055
p value proxy: 0.39303481578826904
result: False
6th run
noise dimension : 25
noise variance: 2222.961669921875
original u-statistic:22.631816864013672
p value proxy: 0.23880596458911896
result: False
7th run
noise dimension : 25
noise variance: 2210.59423828125
original u-statistic:38.55506

In [131]:
####CHANGE HERE#####
n1 = 30000
n2 = 30000
####################
kappa = 5 #number of bins
alpha = 0.3 #privacy level
gamma = 0.05 # significance level
nTests = 200 #number of tests for power estimation
B = 200 # number of permutations

start_time = time.time()

copula_mean_1 = torch.tensor([-1.0, -1.0, -1.0]).to(device)
copula_mean_2 = torch.tensor([1.0, 1.0, 1.0]).to(device)
sigma = torch.tensor([[1.0, 0.5, 0.5], [0.5, 1.0, 0.5],  [0.5, 0.5, 1.0]]).to(device)

tester = LDPTwoSampleTester(device)
generator_X = torch.distributions.multivariate_normal.MultivariateNormal(loc = copula_mean_1, covariance_matrix = sigma)
generator_Y = torch.distributions.multivariate_normal.MultivariateNormal(loc = copula_mean_2, covariance_matrix = sigma)
cdf_calculator = torch.distributions.normal.Normal(loc = 0.0, scale = 1.0)

test_results = torch.empty(nTests)
for rep in range(nTests):
    print(f"{rep+1}th run")
    
    data_x = cdf_calculator.cdf(generator_X.sample((n1,)))
    data_y = cdf_calculator.cdf(generator_Y.sample((n2,)))
    
    
    result_now = tester.run_test_conti_data(B, data_x, data_y,
                                            kappa, alpha, gamma, discrete = False
                                           )
    test_results[rep] = result_now
    print(f"result: {result_now}")
  
print( f"power estimate : { torch.sum(test_results)/nTests }" )
print( f"elapsed time: { time.time() - start_time }" )



1th run
noise dimension : 125
noise variance: 11095.298828125
original u-statistic:37.21411895751953
p value proxy: 0.009950248524546623
result: True
2th run
noise dimension : 125
noise variance: 11120.9072265625
original u-statistic:18.099506378173828
p value proxy: 0.09452736377716064
result: False
3th run
noise dimension : 125
noise variance: 11120.3681640625
original u-statistic:34.653133392333984
p value proxy: 0.004975124262273312
result: True
4th run
noise dimension : 125
noise variance: 11114.7900390625
original u-statistic:15.467294692993164
p value proxy: 0.0746268630027771
result: False
5th run
noise dimension : 125
noise variance: 11105.96484375
original u-statistic:5.542716026306152
p value proxy: 0.2885572016239166
result: False
6th run
noise dimension : 125
noise variance: 11102.81640625
original u-statistic:42.784420013427734
p value proxy: 0.009950248524546623
result: True
7th run
noise dimension : 125
noise variance: 11110.3212890625
original u-statistic:29.0577507019

# 4 dimension copula

In [ ]:
import sys
import torch
sys.stdout = open('copula_4d.txt','w')

device = torch.device('cuda:0')

####CHANGE HERE#####
n1 = 10000
n2 = 10000
####################
kappa = 5 #number of bins
alpha = 0.3 #privacy level
gamma = 0.05 # significance level
nTests = 200 #number of tests for power estimation
B = 200 # number of permutations

start_time = time.time()

copula_mean_1 = torch.tensor([-1.0, -1.0, -1.0, -1.0]).to(device)
copula_mean_2 = torch.tensor([1.0, 1.0, 1.0, 1.0]).to(device)
sigma = torch.tensor([[1.0, 0.5, 0.5, 0.5], [0.5, 1.0, 0.5, 0.5],  [0.5, 0.5, 1.0, 0.5], [0.5, 0.5, 0.5, 1.0]]).to(device)

tester = LDPTwoSampleTester(device)
generator_X = torch.distributions.multivariate_normal.MultivariateNormal(loc = copula_mean_1, covariance_matrix = sigma)
generator_Y = torch.distributions.multivariate_normal.MultivariateNormal(loc = copula_mean_2, covariance_matrix = sigma)
cdf_calculator = torch.distributions.normal.Normal(loc = 0.0, scale = 1.0)

test_results = torch.empty(nTests)
for rep in range(nTests):
    print(f"{rep+1}th run")
    
    data_x = cdf_calculator.cdf(generator_X.sample((n1,)))
    data_y = cdf_calculator.cdf(generator_Y.sample((n2,)))
    
    
    result_now = tester.run_test_conti_data(B, data_x, data_y,
                                            kappa, alpha, gamma, discrete = False
                                           )
    test_results[rep] = result_now
    print(f"result: {result_now}")
  
print( f"power estimate : { torch.sum(test_results)/nTests }" )
print( f"elapsed time: { time.time() - start_time }" )

In [ ]:
n1 = 1000
n2 = 1000
################
kappa = 5
alpha = 0.5
gamma = 0.05
B = 200


start_time = time.time()


tester = LDPTwoSampleTester(device)
generator_X = torch.distributions.beta.Beta(torch.tensor(3.0).to(device), torch.tensor(30.0).to(device))
generator_Y = torch.distributions.beta.Beta(torch.tensor(30.0).to(device), torch.tensor(3.0).to(device))

test_results = torch.empty(100)
for rep in range(100):
    print(f"{rep+1}th run")
    
    result_now = tester.run_test_conti_data(B,
                                            generator_X.sample((n1,)).reshape(-1),
                                            generator_Y.sample((n2,)).reshape(-1),
                                            kappa, alpha, gamma, discrete = False
                                           )
    test_results[rep] = result_now
    print(result_now)
  
print(torch.sum(test_results)/100)
elapsed_time = time.time() - start_time
print(elapsed_time)


In [ ]:




L2distBetaUnif <- function(shape.1, shape.2){
  return(Beta(2 * shape.1 - 1, 2 * shape.2 -1) / (Beta(shape.1, shape.2))^2 - 1)
}

L2distBetaBeta <- function(shape.1.1, shape.1.2, shape.2.1, shape.2.2){
  first.beta.term <- Beta(2 * shape.1.1 - 1, 2 * shape.1.2 -1) / (Beta(shape.1.1, shape.1.2))^2
  second.beta.term <- Beta(2 * shape.2.1 - 1, 2 * shape.2.2 -1) / (Beta(shape.2.1, shape.2.2))^2
  cross.term <- Beta(shape.1.1 + shape.2.1 - 1, shape.1.2 + shape.2.2 -1) / ( (Beta(shape.1.1, shape.1.2)) * (Beta(shape.2.1, shape.2.2)) )
  return(first.beta.term + second.beta.term - 2 * cross.term)
}

Beta <- function(shape.1, shape.2){
  return(gamma(shape.1) * gamma(shape.2) / gamma(shape.1 + shape.2))


In [7]:
data = torch.tensor(
[
[0.19901582, 0.29330425, 0.08031318, 0.27744206],
[0.38371595, 0.07725842, 0.58872328, 0.60947456],
[0.78765378, 0.18596928, 0.20049580, 0.04321161],
[0.60499579, 0.38050702, 0.26301983, 0.58410214],
[0.42093993, 0.42060113, 0.89575178, 0.57233768],
[0.01160462, 0.55440856, 0.50919182, 0.80756614]
]
).to(device)



In [ ]:
torch.linspace(start = (3-1), end = 0, steps = 3, dtype = torch.long)

In [ ]:
data.size(dim = 1)

In [ ]:
torch.sum(data) > 0.05

In [ ]:
data[torch.randperm(data.size(dim=0))]

# 1. test of transform_bin_index

In [34]:
tester = LDPTwoSampleTester(device)
index_1 = tester.transform_bin_index(data, 4)
index_1

tensor([[1, 2, 1, 2],
        [2, 1, 3, 3],
        [4, 1, 1, 1],
        [3, 2, 2, 3],
        [2, 2, 4, 3],
        [1, 3, 3, 4]], device='cuda:0')

In [35]:
generator_X = torch.distributions.beta.Beta(torch.tensor(3.0).to(device), torch.tensor(5.0).to(device))



In [42]:
beta_sample = generator_X.sample((100,))
print(beta_sample)
index_2 = tester.transform_bin_index(beta_sample, 5)
index_2


tensor([0.5314, 0.4275, 0.7200, 0.3289, 0.2905, 0.4236, 0.3151, 0.2569, 0.4084,
        0.2772, 0.3774, 0.2719, 0.2548, 0.1673, 0.3826, 0.2703, 0.4119, 0.5091,
        0.2450, 0.4965, 0.3836, 0.2755, 0.5672, 0.3766, 0.3929, 0.2914, 0.2755,
        0.3891, 0.1551, 0.4460, 0.2921, 0.1289, 0.2629, 0.1929, 0.4320, 0.0801,
        0.5909, 0.4335, 0.1094, 0.0828, 0.3406, 0.2133, 0.4406, 0.3368, 0.1858,
        0.4543, 0.4944, 0.2448, 0.4412, 0.3706, 0.1872, 0.3609, 0.4708, 0.1810,
        0.3931, 0.3136, 0.4998, 0.3496, 0.4917, 0.8134, 0.1857, 0.1171, 0.4542,
        0.3147, 0.4092, 0.2183, 0.6881, 0.1248, 0.5016, 0.4871, 0.3294, 0.3921,
        0.3036, 0.3215, 0.6914, 0.3329, 0.4376, 0.3956, 0.3288, 0.2637, 0.6398,
        0.6667, 0.4865, 0.2412, 0.4486, 0.2631, 0.4048, 0.3683, 0.5045, 0.3206,
        0.1542, 0.2533, 0.6809, 0.3378, 0.3149, 0.6909, 0.3772, 0.3125, 0.3483,
        0.4594], device='cuda:0')


tensor([3, 3, 4, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 1, 2, 2, 3, 3, 2, 3, 2, 2, 3, 2,
        2, 2, 2, 2, 1, 3, 2, 1, 2, 1, 3, 1, 3, 3, 1, 1, 2, 2, 3, 2, 1, 3, 3, 2,
        3, 2, 1, 2, 3, 1, 2, 2, 3, 2, 3, 5, 1, 1, 3, 2, 3, 2, 4, 1, 3, 3, 2, 2,
        2, 2, 4, 2, 3, 2, 2, 2, 4, 4, 3, 2, 3, 2, 3, 2, 3, 2, 1, 2, 4, 2, 2, 4,
        2, 2, 2, 3], device='cuda:0')

# 2.1 Test of TransformMultivariate

In [37]:
multi_1 = tester.TransformMultivariate(index_1, 4);
multi_1

tensor([ 17,  74, 192, 150,  94,  43], device='cuda:0')

In [41]:
multi_2 = tester.TransformMultivariate(index_2, 5)
multi_2

tensor([2, 2, 2, 1, 2, 0, 2, 2, 3, 1, 3, 2, 1, 2, 1, 2, 2, 1, 0, 1, 2, 2, 1, 1,
        0, 2, 2, 1, 1, 1, 2, 0, 0, 1, 2, 1, 2, 1, 1, 1, 1, 3, 2, 1, 1, 2, 1, 0,
        1, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 0, 2, 2, 2, 1, 1, 0, 1, 1, 3, 2,
        1, 2, 2, 2, 1, 1, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 0, 0, 3, 1, 3, 2, 1,
        1, 1, 2, 2], device='cuda:0')

# 3. implementation of TransformOnehot

In [43]:
onehot_1 = tester.TransformOnehot(multi_1, 4**4)
onehot_1

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0')

In [44]:
onehot_2 = tester.TransformOnehot(multi_2, 5)
onehot_2

tensor([[0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],


# implementation of noise.conti

In [ ]:
torch.var(noise_conti(1000, 2, 0.5))


# implementation of noise.discrete

In [ ]:
torch.var(noise_discrete(1000, 2, 0.5))



# implementation of PrivatizeTwoSample

In [ ]:
privatize_twosample(data, alpha = 0.5)

# u-statisitc function

In [ ]:
U_stat_twosample(data, 2)

In [ ]:



      




noise.vairance.theoretic.conti <-function(dim, alpha){
  return(8 * dim / (alpha^2))
}

noise.vairance.theoretic.discrete <-function(dim, alpha){
  p <- exp(-alpha/(2*sqrt(dim)))
  return(2*p/(1-p)^2)
}






In [ ]:
unit_laplace_generator_cpu = torch.distributions.laplace.Laplace(torch.tensor([0.0]), torch.tensor([2**(-1/2)]))
laplace_samples = unit_laplace_generator_cpu.sample(sample_shape = torch.Size([100000000]))
print(laplace_samples.size())
laplace_samples.view(100000, -1)
print(laplace_samples[-1])
print(laplace_samples.shape)

In [ ]:
a = [1,2,3]
a[-1]

In [ ]:
unit_laplace_generator_gpu = torch.distributions.laplace.Laplace(torch.tensor([0.0]).cuda(), torch.tensor([2**(-1/2)]).cuda())

laplace_samples = unit_laplace_generator_gpu.sample(sample_shape = torch.Size([100000000]))


In [ ]:
del(laplace_samples)

In [ ]:
torch.cuda.empty_cache() 